<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C3/W4/ungraded_labs/C3_W4_Lab_2_irish_lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/doantronghieu/DEEP-LEARNING/main/helper_DL.py
!pip install colorama
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':15})
import seaborn           as sns
sns.set()
import helper_DL as helper

# Ungraded Lab: Generating Text from Irish Lyrics

In the previous lab, you trained a model on just a single song. You might have found that the output text can quickly become gibberish or repetitive. Even if you tweak the hyperparameters, the model will still be limited by its vocabulary of only 263 words. The model will be more flexible if you train it on a much larger corpus and that's what you'll be doing in this lab. You will use lyrics from more Irish songs then see how the generated text looks like. You will also see how this impacts the process from data preparation to model training. Let's get started!

## Imports

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow import nn
from tensorflow.keras import layers, losses, optimizers, models, Model
import tensorflow.keras.preprocessing as tfkp

## Building the Word Vocabulary

You will first download the lyrics dataset. These will be from a compilation of traditional Irish songs and you can see them [here](https://github.com/https-deeplearning-ai/tensorflow-1-public/blob/main/C3/W4/misc/Laurences_generated_poetry.txt). 

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course3/irish-lyrics-eof.txt

Next, you will lowercase and split the plain text into a list of sentences:

In [ ]:
# Load the dataset
data = open('./irish-lyrics-eof.txt').read()

# Lower case and split the text
corpus = data.lower().split('\n')

print(corpus)

From here, you can initialize the `Tokenizer` class and generate the word index dictionary:

In [ ]:
# Initialize the Tokenizer class
tokenizer = tfkp.text.Tokenizer()

# Generate the word index dictionary
tokenizer.fit_on_texts(corpus)

# Define the total words. Add 1 for the index `0` which is just the padding token.
total_words = len(tokenizer.word_index) + 1

print(f'Word index dictionary:\n{tokenizer.word_index}\n')
print(f'Total words: {total_words}')

## Preprocessing the Dataset

Next, you will generate the inputs and labels for your model. The process will be identical to the previous lab. The `xs` or inputs to the model will be padded sequences, while the `ys` or labels are one-hot encoded arrays.

In [ ]:
# Initialize the sequences list
input_sequences = []

# Loop over every line
for line in corpus:

	# Tokenize the current line
	token_list = tokenizer.texts_to_sequences([line])[0]

	# Loop over the line several times to generate the subphrases
	for i in range(1, len(token_list)):
		
		# Generate the subphrase
		n_gram_sequence = token_list[:i+1]

		# Append the subphrase the the sequences list
		input_sequences.append(n_gram_sequence)
  
# Get the length of the longest line
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences
input_sequences = np.array(tfkp.sequence.pad_sequences(input_sequences,
                                                       maxlen = max_sequence_len,
													   padding = 'pre'))

# Create inputs and label by splitting the last token in the subphrase
xs     = input_sequences[:, :-1]
labels = input_sequences[:, -1]

# Convert the label into one-hot arrays
ys = tfk.utils.to_categorical(labels, num_classes = total_words)

print(xs.shape, '-', ys.shape)

You can then print some of the examples as a sanity check.

In [ ]:
# Get sample sentence
sentence = corpus[0].split()

# Initialize token list
token_list = []

# Look up the indices of each word and append to the list
for word in sentence:
    token_list.append(tokenizer.word_index[word])

print(f'Corpus: {corpus}\n')
print(f'Sentence: {sentence}\n')
print(f'Token list: {token_list}')

In [ ]:
# Pick element
elem_number = 5

# Print token list and phrase
print(f'Token list:\n{xs[elem_number]}\n')
print(f'Decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}\n')

# Print label
print(f'Ont-hot label:\n{ys[elem_number]}\n')
print(f'Index of label: {np.argmax(ys[elem_number])}')

## Build and compile the Model

Next, you will build and compile the model. We placed some of the hyperparameters at the top of the code cell so you can easily tweak it later if you want.

In [ ]:
# Hyperparameters
EMBEDDING_DIM = 100
LSTM_UNITS    = 150
LEARNING_RATE = 0.001

# Build the model
model = models.Sequential([
    layers.Embedding(total_words, EMBEDDING_DIM, input_length = max_sequence_len - 1),
    layers.Bidirectional(layers.LSTM(LSTM_UNITS)),
    layers.Dense(total_words, activation = nn.softmax)                           
])

# Use categorical crossentropy because this is a multi-class problem
model.compile(loss = losses.categorical_crossentropy,
              optimizer = optimizers.Adam(learning_rate = LEARNING_RATE),
              metrics = ['accuracy'])

model.summary()

## Train the model

From the model summary above, you'll notice that the number of trainable params is much larger than the one in the previous lab. Consequently, that usually means a slower training time. It will take roughly 7 seconds per epoch with the GPU enabled in Colab and you'll reach around 76% accuracy after 100 epochs.

In [ ]:
epochs = 100

# Train the model
history = model.fit(xs, ys, epochs=epochs)

You can visualize the accuracy below to see how it fluctuates as the training progresses.

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

# Visualize the accuracy
plot_graphs(history, 'accuracy')

## Generating Text

Now you can let the model make its own songs or poetry! Because it is trained on a much larger corpus, the results below should contain less repetitions as before. The code below picks the next word based on the highest probability output.

In [ ]:
# Define seed text
seed_text = "help me obi-wan kinobi youre my only hope"

# Total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):

    # Convert the seed text to a token sequence
    token_list = np.squeeze(tokenizer.texts_to_sequences([seed_text]))

    # Pad the sequence
    token_list = tfkp.sequence.pad_sequences([token_list], 
                                                maxlen = max_sequence_len - 1,
                                                padding = 'pre')

    # Feed to the model and get probabilities for each index
    probabilities = np.squeeze(model.predict(token_list))

    # Pick a random number from [1, 2, 3]
    choice = np.random.choice([1, 2, 3])

    # - Sort the probabilities in ascending order and get the random choice from
    # the end of the array
    predicted = np.argsort(probabilities)[-choice]

    # Ignore if index is 0 (just the padding)
    if (predicted != 0):

        # Look up the word associated with the index
        output_word = tokenizer.index_word[predicted]

        # Combine with the seed text
        seed_text += ' ' + output_word

# Print the result
print(seed_text)

Here again is the code that gets the top 3 predictions and picks one at random.

In [ ]:
# Define seed text
seed_text = "help me obi-wan kinobi youre my only hope"

# Total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):

    # Convert the seed text to a token sequence
    token_list = np.squeeze(tokenizer.texts_to_sequences([seed_text]))

    # Pad the sequence
    token_list = tfkp.sequence.pad_sequences([token_list], 
                                                maxlen = max_sequence_len - 1,
                                                padding = 'pre')

    # Feed to the model and get probabilities for each index
    probabilities = np.squeeze(model.predict(token_list))

    # Pick a random number from [1, 2, 3]
    choice = np.random.choice([1, 2, 3])

    # - Sort the probabilities in ascending order and get the random choice from
    # the end of the array
    predicted = np.argsort(probabilities)[-choice]

    # Ignore if index is 0 (just the padding)
    if (predicted != 0):

        # Look up the word associated with the index
        output_word = tokenizer.index_word[predicted]

        # Combine with the seed text
        seed_text += ' ' + output_word

# Print the result
print(seed_text)

## Wrap Up

This lab shows the effect of having a larger dataset to train your text generation model. As expected, this will take a longer time to prepare and train but the output will less likely become repetitive or gibberish. Try to tweak the hyperparameters and see if you get better results. You can also find some other text datasets and use it to train the model here.  